
# 🛡️ Risk Dashboard

A practical dashboard to monitor **portfolio risk**:
- Positions & exposures (asset/sector/currency)
- P/L and drawdown
- Historical **VaR / ES** (non-parametric)
- Factor betas (if factor returns available)
- Stress tests & scenario analysis
- Rolling volatility & correlations

> Drop files into `data/` and the notebook will auto-detect them.  
> If files are missing, it **simulates** reasonable data so the dashboard always runs.


## 0) Parameters

In [ ]:

# Data files (CSV)
PATH_POSITIONS = "data/positions.csv"          # columns: ticker, quantity, avg_price, [sector], [ccy]
PATH_PRICES    = "data/prices.csv"             # wide: date, TICK1, TICK2, ...
PATH_FACTORS   = "data/factors.csv"            # wide: date, MKT, SMB, HML, MOM, ... (optional)
BASE_CCY       = "USD"

# Dashboard knobs
ALPHA = 0.95            # VaR/ES confidence (one-sided)
WINDOW_VOL = 60         # rolling days for volatility
WINDOW_CORR = 60        # rolling days for corr heatmap
SCENARIOS = {
    "Shock -5% All": {"type": "uniform_ret", "ret": -0.05},
    "Rates Up 100bp": {"type": "linear_factor", "beta_key": "DUR", "shock": -0.06},
    "Flight to Quality": {"type": "two_bucket", "winners": ["BOND"], "losers": ["EQT"], "ret_win": 0.01, "ret_lose": -0.03},
}


## 1) Setup & Helpers

In [ ]:

import os, sys, math, json, itertools
from typing import Dict, List, Any, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = "{:,.4f}".format

def ensure_cols(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan
    return df

def pct_change_safe(x: pd.Series) -> pd.Series:
    return x.pct_change().replace([np.inf, -np.inf], np.nan)

def drawdown_curve(nav: pd.Series) -> pd.Series:
    roll_max = nav.cummax()
    return nav/roll_max - 1.0

def historical_var_es(returns: pd.Series, alpha: float=0.95) -> Dict[str, float]:
    r = returns.dropna().sort_values()
    if len(r) == 0:
        return {"var": np.nan, "es": np.nan}
    idx = int((1-alpha) * len(r))
    idx = max(0, min(idx, len(r)-1))
    var = -r.iloc[idx]
    es = -r.iloc[:idx+1].mean() if idx >= 0 else np.nan
    return {"var": float(var), "es": float(es)}

def annualize_vol(ret: pd.Series, freq: int=252) -> float:
    return float(ret.std() * math.sqrt(freq))


## 2) Load Data (with synthetic fallback)

In [ ]:

def load_positions(path=PATH_POSITIONS) -> pd.DataFrame:
    if os.path.exists(path):
        df = pd.read_csv(path)
        df = ensure_cols(df, ["ticker", "quantity", "avg_price", "sector", "ccy"])
        df["ticker"] = df["ticker"].astype(str)
        df["quantity"] = df["quantity"].astype(float)
        df["avg_price"] = df["avg_price"].astype(float)
        df["sector"] = df["sector"].fillna("Unknown")
        df["ccy"] = df["ccy"].fillna(BASE_CCY)
        return df
    # Synthetic positions
    rng = np.random.default_rng(0)
    tickers = [f"T{i:02d}" for i in range(12)]
    qty = rng.integers(100, 2000, size=len(tickers)).astype(float)
    avg_px = rng.uniform(10, 200, size=len(tickers))
    sector = rng.choice(["EQT","BOND","FX","CMD"], size=len(tickers))
    ccy = rng.choice(["USD","EUR","JPY"], size=len(tickers), p=[0.7,0.2,0.1])
    return pd.DataFrame({"ticker": tickers, "quantity": qty, "avg_price": avg_px, "sector": sector, "ccy": ccy})

def load_prices(path=PATH_PRICES, n_days=500, cols: Optional[List[str]]=None) -> pd.DataFrame:
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=["date"]).set_index("date").sort_index()
        return df
    # Synthetic prices following sector-ish correlations
    rng = np.random.default_rng(1)
    if cols is None:
        cols = [f"T{i:02d}" for i in range(12)]
    dates = pd.bdate_range("2023-01-01", periods=n_days)
    market = rng.normal(0.0003, 0.01, size=n_days)
    prices = {}
    for j, c in enumerate(cols):
        sector_idx = j % 4
        sector = rng.normal(0.0002*(sector_idx+1), 0.008, size=n_days)
        idio = rng.normal(0, 0.01, size=n_days)
        ret = market + 0.6*sector + 0.4*idio
        px = 100 * (1 + pd.Series(ret, index=dates)).cumprod().values # type: ignore
        prices[c] = px
    return pd.DataFrame(prices, index=dates)

def load_factors(path=PATH_FACTORS, n_days=500) -> pd.DataFrame:
    if os.path.exists(path):
        return pd.read_csv(path, parse_dates=["date"]).set_index("date").sort_index()
    # Synthetic factor returns (MKT, SMB, HML, MOM, DUR)
    rng = np.random.default_rng(2)
    dates = pd.bdate_range("2023-01-01", periods=n_days)
    data = {
        "MKT": rng.normal(0.0004, 0.01, size=len(dates)),
        "SMB": rng.normal(0.0001, 0.006, size=len(dates)),
        "HML": rng.normal(0.0001, 0.006, size=len(dates)),
        "MOM": rng.normal(0.0002, 0.009, size=len(dates)),
        "DUR": rng.normal(0.0000, 0.004, size=len(dates)),  # duration proxy for rates shock
    }
    return pd.DataFrame(data, index=dates)

pos = load_positions()
prices = load_prices(cols=pos["ticker"].tolist())
factors = load_factors(n_days=len(prices))

prices.head(), pos.head(), factors.head()


## 3) Derived Series: Returns, NAV, PnL

In [ ]:

returns = prices.pct_change().replace([np.inf, -np.inf], np.nan).fillna(0.0)

# Mark-to-market current prices (last row)
last_px = prices.iloc[-1]
mkt_value = (last_px.reindex(pos["ticker"]) * pos["quantity"].values)
pos["market_value"] = mkt_value.values
pos["exposure_%"] = pos["market_value"] / pos["market_value"].sum()

# Portfolio NAV (start at 1.0)
weights = pos.set_index("ticker")["exposure_%"].reindex(prices.columns).fillna(0.0)
port_ret = (returns * weights).sum(axis=1)
nav = (1 + port_ret).cumprod()
dd = drawdown_curve(nav)

# Daily PnL in currency terms (assume base USD; prices as USD)
notional = pos["market_value"].sum()
pnl = port_ret * notional

pos.sort_values("market_value", ascending=False).head(10)


## 4) Exposures

In [ ]:

expo_sector = pos.groupby("sector")["market_value"].sum().sort_values(ascending=False)
expo_ccy    = pos.groupby("ccy")["market_value"].sum().sort_values(ascending=False)

print("Sector exposures:")
display(expo_sector.to_frame("mv"))

print("Currency exposures:")
display(expo_ccy.to_frame("mv"))


## 5) NAV, Drawdown, PnL

In [ ]:

plt.figure(figsize=(10,4))
nav.plot()
plt.title("Portfolio NAV")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3))
dd.plot()
plt.title("Drawdown")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3))
pnl.plot()
plt.title("Daily PnL")
plt.tight_layout()
plt.show()


## 6) Historical VaR / ES

In [ ]:

risk = historical_var_es(port_ret, alpha=ALPHA)
print(f"{int(ALPHA*100)}% 1-day VaR: {risk['var']:.4%}")
print(f"{int(ALPHA*100)}% 1-day ES : {risk['es']:.4%}")
print(f"Ann. Vol (port): {annualize_vol(port_ret):.2%}")


## 7) Rolling Volatility & Correlation

In [ ]:

roll_vol = returns.rolling(WINDOW_VOL).std().iloc[-1] * np.sqrt(252)

plt.figure(figsize=(10,3))
roll_vol.sort_values(ascending=False).plot(kind="bar")
plt.title(f"Rolling {WINDOW_VOL}d Volatility (annualized)")
plt.tight_layout()
plt.show()

# Correlation heatmap of top-N notional names
top_ticks = pos.sort_values("market_value", ascending=False)["ticker"].head(12).tolist()
corr = returns[top_ticks].tail(WINDOW_CORR).corr()

plt.figure(figsize=(6,5))
plt.imshow(corr.values, vmin=-1, vmax=1)
plt.xticks(range(len(top_ticks)), top_ticks, rotation=45, ha='right')
plt.yticks(range(len(top_ticks)), top_ticks)
plt.colorbar()
plt.title(f"Correlation (last {WINDOW_CORR}d) — Top Exposures")
plt.tight_layout()
plt.show()


## 8) Factor Betas (OLS, quick)

In [ ]:

def regress_beta(y: pd.Series, X: pd.DataFrame) -> pd.Series:
    yv = y.values.reshape(-1,1) # type: ignore
    Xv = np.column_stack([np.ones(len(X))] + [X[c].values for c in X.columns]) # type: ignore
    beta = np.linalg.lstsq(Xv, yv, rcond=None)[0].flatten()
    return pd.Series(beta[1:], index=X.columns)  # exclude intercept

# Portfolio factor exposure approximated by weighted asset returns
Y = (returns[top_ticks] * weights.reindex(top_ticks)).sum(axis=1).dropna().align(factors, join="inner")[0]
X = factors.loc[Y.index]
betas = regress_beta(Y, X)

display(betas.to_frame("beta"))

plt.figure(figsize=(8,3))
betas.sort_values(ascending=False).plot(kind="bar")
plt.title("Estimated Factor Betas (portfolio)")
plt.tight_layout()
plt.show()


## 9) Stress Tests & Scenarios

In [ ]:

def run_scenarios(pos: pd.DataFrame, last_prices: pd.Series, scenarios: Dict[str, Dict[str, Any]]) -> pd.DataFrame:
    rows = []
    mv = (last_prices.reindex(pos['ticker']) * pos['quantity'].values)
    for name, sc in scenarios.items():
        if sc["type"] == "uniform_ret":
            ret = sc["ret"]
            pnl = (mv * ret).sum()
        elif sc["type"] == "linear_factor":
            # Requires betas; map a beta key to a shock return
            shock = sc.get("shock", -0.05)
            # Map each ticker to beta via sector proxy (simplified)
            # EQT -> behaves like MKT beta 1.0, BOND -> DUR beta 1.0, FX/CMD -> 0.3
            map_beta = pos["sector"].map({"EQT": 1.0, "BOND": 1.0, "FX": 0.3, "CMD": 0.3}).fillna(0.5)
            pnl = float((mv * map_beta.values * shock).sum())
        elif sc["type"] == "two_bucket":
            winners = pos["sector"].isin(sc.get("winners", []))
            losers  = pos["sector"].isin(sc.get("losers", []))
            ret = np.where(winners, sc.get("ret_win", 0.01), np.where(losers, sc.get("ret_lose", -0.03), 0.0))
            pnl = float((mv * ret).sum())
        else:
            pnl = np.nan
        rows.append({"scenario": name, "pnl": pnl})
    return pd.DataFrame(rows).set_index("scenario")

sc_df = run_scenarios(pos, last_px, SCENARIOS)
display(sc_df)

plt.figure(figsize=(8,3))
sc_df["pnl"].plot(kind="bar")
plt.title("Scenario PnL")
plt.tight_layout()
plt.show()


## 10) Limits & SLO Hooks

In [ ]:

LIMITS = {
    "Max Leverage": 4.0,
    "Max Gross notional (USD)": 5_000_000.0,
    "Max DD": -0.10,
    "Target Vol (ann)": 0.08,
}

violations = []
gross = pos["market_value"].sum()
if dd.min() < LIMITS["Max DD"]:
    violations.append(("Drawdown", float(dd.min()), LIMITS["Max DD"]))
if annualize_vol(port_ret) > LIMITS["Target Vol (ann)"]:
    violations.append(("Volatility", float(annualize_vol(port_ret)), LIMITS["Target Vol (ann)"]))

print("Limits:")
print(LIMITS)
print("\nPotential Violations:")
for v in violations:
    print("-", v[0], "=", v[1], "limit", v[2])

# Placeholder SLO metrics you can export for Prometheus
SLO = {
    "risk_var_1d": risk["var"],
    "risk_es_1d": risk["es"],
    "nav": float(nav.iloc[-1]),
    "drawdown": float(dd.iloc[-1]),
}
SLO
